# Session 2

## Implementing a 2D simulation of Active Brownian Particles (ABP) in C++

In the second session of this tutorial we port the Python code developed in Session 1 to C++. We keep the structure and naming the same as in the Python version and only make changes where necessary to set the sage for the GPU implementation in Session 3.

### Brief overview of the design layout

The image below show key components of a particle based simulation code and how they interact with each other: 

<div align="center">
    
<img src="./layout.png" style="width: 800px;"/>
</div>

Let us briefly focus on the *Evolver* class. Its task is to evolve the system by a single time step, i.e., to propagate the dynamics for $\delta t$. In particular, its task is to:

1. Ensure that the neighbor list is up to date;
2. Perform the integration pre-step;
3. Compute all forces and torques on each particle;
4. Perform the integration post-step;
5. Apply periodic boundary conditions.

In our design, one can have multiple force and torque laws apply in the system. In addition, multiple integrators can act in the same system. 


Here we focus on the item number 3, and in particular the fact that in a given system one could have multiple types of interactions. 

For example, in our case, we apply the self-propulsion force $\mathbf{F}^\text{sp}_i = \alpha \mathbf{n}_i$ on each particle. In addition, each particle experiences soft-core repulsion due to overlaps with its neighbors, i.e. $\mathbf{F}_{i}^{\text{elastic}} = \sum_j k\left(2a - r_{ij}\right)\hat{\mathbf{r}}_{ij}$ for $r_{ij} \le 2a$, with the same meaning of parameters as used in Session 1. In our implementation, each term in the total force is implemented as a separate class. The self-propulsion term is handled by the *SelfPropulsion* class, while the soft-core repulsion is implemented in the *HarmonicForce* class.

The *Evolver* class has member variable called *force_computes* that is a Python list that stores all different types of interactions used in a given simulation. To be a bit more technical, if we want to add self-propulsion and soft-core repulsion to our system, we simply create two objects (instances), one for each of the two ""force type" classes and append them to the *force_computes* list of the *Evolver* class. 

The *evolve* function of the *Evolver* class then **simply** iterates over all elements of the *force_computes* list and calls the appropriate *compute* function for each of element.

It is not a coincidence that we put the word **simply** in bold. Due to Python's expressive and powerful data structure this was all we needed to do. However, a lot of things are happening under the hood here, which we need to understand if we are to switch to C++. Code written in C++ is in general much faster than the same code written in Python. This, however, comes at the expense of lot of things that, colloquially put, just work in Python, need to be explicitly implemented in C++.

In order to understand how to port to C++ the same functionality that allows us to split different types of forces into separate classes, we need to introduce the concepts of *class inheritance* and *smart pointers*.